# Classifier training

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import scipy.stats as sm
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# Loading the data

In [2]:
# names=['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
df = pd.read_csv('data/out.csv')

df.head()

,age,sex,chest_pain,rest_blood_pressure,cholestorol,fast_blood_sugar,rest_ecg,max_heart_rate,exercise_angina,st_depression,...,age_young,age_middle,bp_1,bp_2,bp_3,bp_4,ch_1,ch_2,ch_3,ch_4
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,...,True,False,False,False,False,True,False,True,False,False
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,...,True,False,False,False,False,True,False,False,False,True
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,...,True,False,True,False,False,False,False,True,False,False
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,...,False,False,False,False,True,False,False,False,True,False
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,...,False,False,False,False,True,False,True,False,False,False


In [3]:
# list(df['rest_blood_pressure'])

In [4]:
# y = df.iloc[:, -1].values
# X = df.iloc[:, :-1]

y = df["target"]
X = df.drop("target", axis=1)

## Split data
Splitting the data into two sets (`train` and `test`) means that the test set can be used as a new, unbiased dataset to evaluate the performance of the model against. This also helps detect the phenomenon of overfitting, where a model is extremely tailored to the training dataset, and therefore performs poorly on unseen datasets.

In this case, 80% of the samples will go to the training set, and the remaining 20 will go to the test set.

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("y_train:", y_train.shape, "y_test:", y_test.shape)

X_train: (736, 24) X_test: (184, 24)
y_train: (736,) y_test: (184,)


### Dummy Variables


In [6]:
df_train = pd.get_dummies(data=X_train, drop_first=True)
df_test = pd.get_dummies(data=X_test, drop_first=True)

## Scale data
Variables with much broader ranges of values might disproportionately effect the final model, so standardising the data ensures that all features are proportionately weighted. 

The formula used for feature standardisation transforms each feature to have zero-mean and unit-variance:

$ \large{x' = \frac{x - \overline{x}}{\sigma}} $

Where $x$ is the original feature vector, $ \bar{x}=average(x)$ is the mean of that feature vector, and $\sigma$ is its standard deviation.

In [7]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  """


# Training the classifiers
The following models will be compared:

Using `sklearn`: 
- Decision tree 
- Support Vector Machine
- Logistic Regression
- Random Forest

Using `Tensorflow`:
- Artificial Neural Network

## Decision Tree classifier

In [8]:
def get_accuracy():
    print("Sklearn accuracy:", sklearn.metrics.accuracy_score(y_test, y_pred))
    print("Sklearn recall:", sklearn.metrics.recall_score(y_test, y_pred, average='micro'))
    print("\nConfusion matrix (predicted vs actual)")
    print(pd.DataFrame(np.c_[sklearn.metrics.confusion_matrix(y_test, y_pred)]))  

In [9]:
from sklearn.tree import DecisionTreeClassifier  

# Create a decision tree Classifier
classifier = DecisionTreeClassifier()  

# Fit the model
classifier.fit(X_train, y_train)  

# Predict the response for test dataset
y_pred = classifier.predict(X_test)  

# Get accuracy results
get_accuracy()

Sklearn accuracy: 0.41304347826086957
Sklearn recall: 0.41304347826086957

Confusion matrix (predicted vs actual)
    0   1  2   3  4
0  57  12  6   9  1
1  15   6  6  10  2
2   3  10  9   7  1
3   1   4  6   3  2
4   1   6  2   4  1


## Support Vector Machine

In [10]:
from sklearn.svm import SVC

# Create an SVM Classifier
classifier = SVC(kernel='linear') # Linear Kernel

# Fit the model
classifier.fit(X_train, y_train)  

# Train the model and predict against the validation set
y_pred = classifier.predict(X_test)  

# Get accuracy results
get_accuracy()

Sklearn accuracy: 0.532608695652174
Sklearn recall: 0.532608695652174

Confusion matrix (predicted vs actual)
    0   1  2   3  4
0  72  11  1   1  0
1  18  15  0   6  0
2   7   8  4  11  0
3   2   6  1   7  0
4   3   2  3   6  0


## Logistic regression

In [11]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')

# Fit the model
classifier.fit(X_train, y_train)  

# Predict the response for test dataset
y_pred = classifier.predict(X_test)  

# Get accuracy results
get_accuracy()

Sklearn accuracy: 0.5434782608695652
Sklearn recall: 0.5434782608695652

Confusion matrix (predicted vs actual)
    0   1  2  3  4
0  74   8  2  1  0
1  19  15  1  4  0
2   8   9  3  9  1
3   2   4  1  8  1
4   2   4  1  7  0


## Neural Network

The Neural Network was built with trial and error, until the best performing network was found. 
It contains:
- 13 input nodes, which correspond to the 13 input features
- 6 nodes in the first hidden layer, which uses the `ReLU` activation function and the `uniform` bias initialiser
- 13 nodes in the second hidden layer, which also uses the `ReLU` activation function
- 1 output layer with 4 nodes, which uses the `Softmax` activation function. This layer outputs a prediction of the level of severity of heart disease (`0` = absence, `1-4` = presence) 
- Categorical cross entropy (`categorical_crossentropy`) for calculating the loss function
- Adam (`adam`) algorithm for optimising the loss function

<img src="data/nn.png" width="600" />

In [12]:
df = pd.read_csv('data/out.csv')

drop = ['target', 'age_mature', 'age_young', 'age_middle', 'bp_1', 'bp_2', 'bp_3', 'bp_4',
       'ch_1', 'ch_2', 'ch_3', 'ch_4']

y = df["target"]
X = df.drop(drop, axis=1)

In [13]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False,  categories='auto')
y = encoder.fit_transform(y.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print("X_train:", X_train.shape, "X_test:", X_test.shape)
print("y_train:", y_train.shape, "y_test:", y_test.shape)

X_train: (736, 13) X_test: (184, 13)
y_train: (736, 5) y_test: (184, 5)


In [14]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [21]:
import tensorflow as tf
from tensorflow import keras

classifier = keras.Sequential()

# adding the input layer (input_dim) and the first hidden layer
classifier.add(keras.layers.Dense(6, bias_initializer = 'uniform', activation = 'relu', input_dim = X_train.shape[1], name='hidden_1'))

# adding the second and third hidden layers
classifier.add(keras.layers.Dense(13, activation='relu', name='hidden_2'))

# adding the output layer
classifier.add(keras.layers.Dense(y_train.shape[1], activation='softmax', name='output'))

In [22]:
# compiling the neural network
# classifier.compile(optimizer = 'sgd', loss = 'mean_squared_error', metrics = ['recall'])
classifier.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

In [23]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden_1 (Dense)             (None, 6)                 84        
_________________________________________________________________
output (Dense)               (None, 5)                 35        
Total params: 119
Trainable params: 119
Non-trainable params: 0
_________________________________________________________________


In [24]:
# fitting the model 
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
736/736 [==============================] - 0s 607us/step - loss: 1.7908 - acc: 0.1658
Epoch 2/100
736/736 [==============================] - 0s 160us/step - loss: 1.6324 - acc: 0.2201
Epoch 3/100
736/736 [==============================] - 0s 416us/step - loss: 1.5242 - acc: 0.3315
Epoch 4/100
736/736 [==============================] - 0s 267us/step - loss: 1.4420 - acc: 0.4171
Epoch 5/100
736/736 [==============================] - 0s 159us/step - loss: 1.3759 - acc: 0.4715
Epoch 6/100
736/736 [==============================] - 0s 143us/step - loss: 1.3218 - acc: 0.4959
Epoch 7/100
736/736 [==============================] - 0s 161us/step - loss: 1.2777 - acc: 0.5068
Epoch 8/100
736/736 [==============================] - 0s 180us/step - loss: 1.2418 - acc: 0.5095
Epoch 9/100
736/736 [==============================] - 0s 139us/step - loss: 1.2143 - acc: 0.5204
Epoch 10/100
736/736 [==============================] - 0s 167us/step - loss: 1.1922 - acc: 0.5231
Epoch 11/100
736/73

736/736 [==============================] - 0s 159us/step - loss: 1.0162 - acc: 0.5897
Epoch 84/100
736/736 [==============================] - 0s 157us/step - loss: 1.0153 - acc: 0.5938
Epoch 85/100
736/736 [==============================] - 0s 146us/step - loss: 1.0151 - acc: 0.5924
Epoch 86/100
736/736 [==============================] - 0s 137us/step - loss: 1.0148 - acc: 0.5924
Epoch 87/100
736/736 [==============================] - 0s 145us/step - loss: 1.0146 - acc: 0.5910
Epoch 88/100
736/736 [==============================] - 0s 128us/step - loss: 1.0141 - acc: 0.5897
Epoch 89/100
736/736 [==============================] - 0s 121us/step - loss: 1.0135 - acc: 0.5910
Epoch 90/100
736/736 [==============================] - 0s 141us/step - loss: 1.0133 - acc: 0.5910
Epoch 91/100
736/736 [==============================] - 0s 122us/step - loss: 1.0129 - acc: 0.5870
Epoch 92/100
736/736 [==============================] - 0s 130us/step - loss: 1.0124 - acc: 0.5883
Epoch 93/100
736/736 [=

In [25]:
results = classifier.evaluate(X_test, y_test)

print('Final test set loss: {:4f}'.format(results[0]))
print('Final test set accuracy: {:4f}'.format(results[1]))

184/184 [==============================] - 0s 685us/step
Final test set loss: 1.274317
Final test set accuracy: 0.461957


In [32]:
from sklearn.metrics import confusion_matrix

y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(cm)

[[62  6  3  6  0]
 [15 15  6  7  0]
 [ 6 15  2  4  0]
 [ 4 10  7  6  1]
 [ 0  1  4  4  0]]


In [30]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [31]:
print_cm(cm, ["0", "1", "2", "3", "4"])

              0     1     2     3     4 
        0  62.0   6.0   3.0   6.0   0.0 
        1  15.0  15.0   6.0   7.0   0.0 
        2   6.0  15.0   2.0   4.0   0.0 
        3   4.0  10.0   7.0   6.0   1.0 
        4   0.0   1.0   4.0   4.0   0.0 


In [ ]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(classifier, ".")